In [17]:
! pip install librosa numpy pandas demucs soundfile pyfluidsynth
! sudo yum install libsndfile

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Loaded plugins: dkms-build-requires, extras_suggestions, langpacks, priorities,
              : update-motd, versionlock
amzn2-core                                               | 3.7 kB     00:00     
https://download.docker.com/linux/centos/2/x86_64/stable/repodata/repomd.xml: [Errno 14] HTTPS Error 404 - Not Found
Trying other mirror.
neuron                                                   | 2.9 kB     00:00     
61 packages excluded due to repository priority protections
Package libsndfile-1.0.25-12.amzn2.1.x86_64 already installed and latest version
Nothing to do


In [1]:
import librosa
import numpy as np
import os
import pandas as pd
import io
from pathlib import Path
from shutil import rmtree
import subprocess as sp


# Source Separation using Demucs

In [2]:
model = "htdemucs"
extensions = ["mp3", "wav", "ogg", "flac"]  # we will look for all those file types.
two_stems = None   # only separate one stems from the rest, for instance

# Options for the output audio.
mp3 = True
mp3_rate = 320
float32 = False  # output as float 32 wavs, unsused if 'mp3' is True.
int24 = False    # output as int24 wavs, unused if 'mp3' is True.

In [3]:
from dataclasses import dataclass

@dataclass
class DeMixedAudio():
    sample_rate: int
    bass : np.ndarray
    drums : np.ndarray
    other : np.ndarray
    vocals : np.ndarray

- Does the model work with different sample rates?
- Does the model output have the same sample rate as the input?
- If so TODO: find a way to find the input native sample rate of the audio and use this to re-load the stems

In [4]:
file_name = os.path.basename(os.path.join(os.getcwd(),'..','resources/test_file.wav'))
sp.run(["python3", "-m", "demucs.separate", "-o", "../resources/tmp", "-n", model, os.path.join(os.getcwd(),'..','resources/test_file.wav')])
print("Demucs done, loading files")

ffprobe: error while loading shared libraries: libopenh264.so.5: cannot open shared object file: No such file or directory


Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /home/ec2-user/SageMaker/scalable-asset-generation/resources/tmp/htdemucs
Separating track /home/ec2-user/SageMaker/scalable-asset-generation/src/../resources/test_file.wav


100%|████████████████████████████████████████████████████████████████████████| 128.7/128.7 [00:33<00:00,  3.86seconds/s]


Demucs done, loading files


In [5]:
def get_stems(path_to_file,model='htdemucs',):
    file_name = os.path.basename(path_to_file)
    sp.run(["python3", "-m", "demucs.separate", "-o", "../resources/tmp", "-n", model, path_to_file])
    print("Demucs done, loading files")
    stems={}
    for stem in ['bass','drums','other','vocals']:
        stems[stem] = librosa.load(os.path.join("../resources/tmp/htdemucs",file_name[:file_name.find('.')],f"{stem}.wav"), sr=44100)[0]
    rmtree(f"../resources/tmp/htdemucs/{file_name[:file_name.find('.')]}")
    return DeMixedAudio(44100, bass=stems['bass'], drums=stems['drums'], other=stems['other'], vocals=stems['vocals'])#stems

In [6]:
song = get_stems(os.path.join(os.getcwd(),'..','resources/test_file.wav'))

ffprobe: error while loading shared libraries: libopenh264.so.5: cannot open shared object file: No such file or directory


Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /home/ec2-user/SageMaker/scalable-asset-generation/resources/tmp/htdemucs
Separating track /home/ec2-user/SageMaker/scalable-asset-generation/src/../resources/test_file.wav


100%|████████████████████████████████████████████████████████████████████████| 128.7/128.7 [00:06<00:00, 19.55seconds/s]


Demucs done, loading files


In [7]:
song

DeMixedAudio(sample_rate=44100, bass=array([-3.8146973e-04, -7.6293945e-05,  9.7656250e-04, ...,
        4.8828125e-04,  4.5776367e-04,  4.5776367e-04], dtype=float32), drums=array([-0.0015564 , -0.00050354,  0.00028992, ...,  0.00030518,
        0.00032043,  0.00032043], dtype=float32), other=array([-0.00183105,  0.00086975, -0.00340271, ...,  0.00015259,
        0.00027466,  0.0002594 ], dtype=float32), vocals=array([-0.00064087,  0.00028992,  0.00042725, ...,  0.00042725,
        0.00039673,  0.00039673], dtype=float32))

# Pop2Piano

In [8]:

!apt-get install -y fluidsynth
!git clone https://github.com/sweetcocoa/pop2piano/
!cd pop2piano
!pip install pretty-midi==0.2.9 omegaconf==2.1.1 youtube-dl==2021.12.17 transformers==4.16.1 pytorch-lightning==1.8.4 essentia==2.1b6.dev609 note-seq==0.0.3 pyFluidSynth==1.3.0
!wget https://github.com/sweetcocoa/pop2piano/releases/download/dpi_2k_epoch/model-1999-val_0.67311615.ckpt

/bin/sh: apt-get: command not found
Cloning into 'pop2piano'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 58 (delta 19), reused 51 (delta 15), pack-reused 0
Receiving objects: 100% (58/58), 128.16 KiB | 18.31 MiB/s, done.
Resolving deltas: 100% (19/19), done.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 42.2 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 125.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 119.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.0/800.0 kB 103.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 8.2 MB/s eta 0:00:0000:01m0:01m
     ━

In [10]:
import os 
import sys
sys.path.append("pop2piano")
import glob
import random

import torch
import torchaudio
import librosa
import numpy as np
import pandas as pd
import IPython.display as ipd
import soundfile as sf
# from google.colab import files

from tqdm.auto import tqdm
from omegaconf import OmegaConf
import note_seq

from utils.dsp import get_stereo
from utils.demo import download_youtube
from transformer_wrapper import TransformerWrapper
from midi_tokenizer import MidiTokenizer, extrapolate_beat_times
from preprocess.beat_quantizer import extract_rhythm, interpolate_beat_times

[   INFO   ] MusicExtractorSVM: no classifier models were configured by default


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
config = OmegaConf.load("pop2piano/config.yaml")
wrapper = TransformerWrapper(config)
wrapper = wrapper.load_from_checkpoint("model-1999-val_0.67311615.ckpt", config=config).to(device)
model = "dpipqxiy"

composer='composer1'
wrapper.eval()

In [19]:
pm, composer, mix_path, midi_path = wrapper.generate(
    audio_path=os.path.join(os.getcwd(),'..','resources/test_file.wav'), 
    composer=composer, 
    model=model,
    show_plot=False, 
    save_midi=True, 
    save_mix=False, 
)
# note_seq.plot_sequence(note_seq.midi_to_note_sequence(pm))

Loading BokehJS ...

In [20]:
! pwd

/home/ec2-user/SageMaker/scalable-asset-generation/src


In [ ]:
! 